# Examples of using CAD_to_OpenMC and various ways of manipulating the material tags in the final geometry file
In this notebook we will show some different ways with which it is possible to manipulate the material tags that are written to the output h5m-geometry files.
First step is of course to enable CAD_to_OpenMC, second to create the meshed files, thirdly we will create OpenMC-simulations, to showcase how to do it.

# Install cad-to-openmc in this notebook, if it is not already available
The branch which picks a moba version is needed because there is not yet an official build of the moab-database library published through PyPi (although this is scheduled for next release).

In [1]:
import sys
import subprocess as sp
pyv = f"{sys.version_info.major}{sys.version_info.minor}"
try:
    import CAD_to_OpenMC.assembly as ab
    print("Found installed CAD_to_OpenMC.")
except ImportError as e:
    wheelstring = f"https://github.com/shimwell/wheels/raw/refs/heads/main/moab/moab-wheels-ubuntu-latest/moab-5.5.1-cp{pyv}-cp{pyv}-manylinux_2_28_x86_64.whl"
    sp.run([sys.executable,"-m","pip","install",wheelstring])
    sp.run([sys.executable,"-m","pip","install","cad-to-openmc"])
    import CAD_to_OpenMC.assembly as ab

Found installed CAD_to_OpenMC.


In [17]:
import importlib.util
if not importlib.util.find_spec("openmc"):
    try:
        sp.run([sys.executable,"-m","pip","install", "openmc"])
    except:
        wheelstring = f"https://github.com/shimwell/wheels/raw/refs/heads/main/openmc/openmc-0.15.1.dev0-cp{pyv}-cp{pyv}-linux_x86_64.whl"
        sp.run([sys.executable,"-m","pip","install",wheelstring])

## Material tags in the output file
By default, CAD_to_OpenMC uses the 1st part of the part name defined in the CAD-model as a material tag for use with particle transport. By 1st part is understood any alphanumeric sequence up to aa whitespace, underscore, or "@". This means that the part names "steel pipe", "steel_flange", "steel@wall" will all be tagged as being made of the material "steel". It is possible to change this behaviour, though.

The file ```pincell1.step``` contains a simple description of a pin with 4 distinct parts tagged as "h2o outer", "h2o inner", "zirconium", and "uo2". While it is possible to define two identical materials with different names in OpenMC, this is often inconvenient. Using a python dictionary approach we may use regular expressions to map material tags to part names. Like this:

In [ ]:
tags={'h2.*':'water','zirconium':'Zi','uo[0-9]':'uranium_oxide'}
A = ab.Assembly(['examples/step_files/'])
A.run(h5m_filename="retagged0.h5m", tags=tags)

This will create a discretized geometry with material tags: "water","Zi", and "uranium_oxide", where "water" is applied to both inner and outer water parts.

Another option to manipulate the material labels is to edit the default delimiter for splitting the original label. Say you want to make a difference between inner and outer water. This could be targeted as:

In [ ]:
B = ab.Assembly(['examples/step_files/pincell1.step'])
B.set_tag_delim('_')
B.run(h5m_filename="retagged1.h5m")

The result would be that the tags propagated to the output file are identical to the part names in full: ```"h2o outer", "h2o inner", "zirconium", "uo2"```.

Lastly, a third possibility is to smiply supply a list of material label to be applied to the parts in the sequence they appear in the step-file.

In [ ]:
stags = ["mat0", "mat1", "mat2", "mat3"]
C = ab.Assembly(['examples/step_files/pincell1.step'])
C.run(h5m_filename="retagged2.h5m", sequential_tags=stags)